In [1]:
!apt update
!pip install gym-super-mario-bros
!pip install gym
!apt-get install python-opengl -y
!apt-get install xvfb --fix-missing
!pip install xvfbwrapper
!apt install xvfb -y
!pip install gym[atari]
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install x11-utils
!apt install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [857 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sour

In [0]:
import xvfbwrapper
from pyvirtualdisplay import Display
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image

import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import os
import base64
from IPython.display import HTML
from functools import reduce

from IPython import display as ipythondisplay
display = Display(visible=0, size=(1400, 900))
display.start()

folder = "./drive/My Drive/601R/project_2/recordings/"
p_model_path = "./drive/My Drive/601R/project_2/models/policy1.pt"
v_model_path = "./drive/My Drive/601R/project_2/models/value1.pt"
step_graph = "./drive/My Drive/601R/project_2/graphs/steps1.png"
v_graph = "./drive/My Drive/601R/project_2/graphs/v_loss1.png"
p_graph = "./drive/My Drive/601R/project_2/graphs/p_loss1.png"
img_size = 32

In [0]:
# env = gym_super_mario_bros.make('SuperMarioBros-v0')
# env = JoypadSpace(env, SIMPLE_MOVEMENT)
# env = gym.wrappers.Monitor(env, folder,force=True)

# state = env.reset()
# env.render()
# img_size = 64

# state = env.reset()
# action_dim = env.action_space.n


# env.get_keys_to_action()
# plt.imshow(env.render('rgb_array'))

# x = resize(state).unsqueeze(0)
# policy_net = Network(state_dim=img_size,action_dim=action_dim)
# value_net = Network(state_dim=img_size)

# y = value_net(x)
# print(y)
# print(env.action_space.sample())

# done = True
# for step in range(1):
#     if done:
#         state = env.reset()
#     state, reward, done, info = env.step(env.action_space.sample())
    # env.render('rgb_array')
    # plt.imshow(env.render('rgb_array'))
    # plt.show()

In [0]:
class Network(nn.Module):
    
  def __init__(self, state_dim, action_dim=1):
    """
    Takes in a state and decides which action to take based on the given state
    """
    super(Network, self).__init__()
    self.action_dim = action_dim
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 16, kernel_size=3, stride=1)
    self.bn3 = nn.BatchNorm2d(16)

    self.linear = nn.Linear(256, action_dim)
    self.soft_max =nn.Softmax(dim=1)

      
  def forward(self, x):
      x = F.relu(self.bn1( self.conv1(x)))
      x = F.relu(self.bn2(self.conv2(x)))
      x = F.relu(self.bn3(self.conv3(x)))
      if self.action_dim == 1:
          return self.linear(x.view(x.size(0), -1))
      else:
          return self.soft_max(self.linear(x.view(x.size(0), -1)))


class AdvantageDataset(Dataset):   
  """
  Takes care of the Data set for the advantages
  """
  def __init__(self, experience):                                                                                                                 
    super(AdvantageDataset, self).__init__()                                                                                                    
    self._exp = experience                                                                                                                      
    self._num_runs = len(experience)                                                                                                            
    self._length = reduce(lambda acc, x: acc + len(x), experience, 0)                                                                           

  def __getitem__(self, index):                                                                                                                   
    idx = 0                                                                                                                                     
    seen_data = 0                                                                                                                               
    current_exp = self._exp[0]                                                                                                                  
    while seen_data + len(current_exp) - 1 < index:                                                                                             
        seen_data += len(current_exp)                                                                                                           
        idx += 1                                                                                                                                
        current_exp = self._exp[idx]                                                                                                            
    chosen_exp = current_exp[index - seen_data]                                                                                                 
    return chosen_exp[0], chosen_exp[4]                                                                                                         

  def __len__(self):                                                                                                                              
    return self._length                                                                                                                         
  
class PolicyDataset(Dataset):   
  """
  Takes care of the policys 
  """
  def __init__(self, experience):                                                                                                                 
    super(PolicyDataset, self).__init__()                                                                                                       
    self._exp = experience                                                                                                                      
    self._num_runs = len(experience)                                                                                                            
    self._length = reduce(lambda acc, x: acc + len(x), experience, 0)                                                                           

  def __getitem__(self, index):                                                                                                                   
    idx = 0                                                                                                                                     
    seen_data = 0                                                                                                                               
    current_exp = self._exp[0]                                                                                                                  
    while seen_data + len(current_exp) - 1 < index:                                                                                             
        seen_data += len(current_exp)                                                                                                           
        idx += 1                                                                                                                                
        current_exp = self._exp[idx]                                                                                                            
    chosen_exp = current_exp[index - seen_data]                                                                                                 
    return chosen_exp                                                                                                                           

  def __len__(self):                                                                                                                              
    return self._length 
  

class Calculate:
  """
  Calculates the the returns and the advantages for each rollout
  """
  def __init__(self):
    pass
  
  def returns(self, rollouts, gamma):
    for i, rollout in enumerate(rollouts):
      projected_reward = 0
      for j in range(len(rollout))[::-1]:
        state, probabilities, action_id, reward = rollout[j]
        projected_reward = reward + (gamma * projected_reward)
        rollout[j] = [state, probabilities, action_id, reward, projected_reward]
  
  def advantages(self, rollouts, value):
    for i, rollout in enumerate(rollouts):
      for j, expererience in enumerate(rollout):
        expected_reward = expererience[4]
        state = expererience[0]
        advantage = expected_reward - value(torch.from_numpy(np.ascontiguousarray(state)).float().unsqueeze(0).cuda()).cpu().squeeze(0).item()
        rollout[j] = expererience + [advantage]

resize = T.Compose([T.ToPILImage(),
                        T.Resize((img_size,img_size), interpolation=Image.CUBIC),
                        T.ToTensor()])

def format_state(img):
    return resize(img).unsqueeze(0)


In [0]:
def train(e, env, policy_net, policy_optim, policy_loss, value_net,value_optim, value_loss):
    
    calculate = Calculate()
    action_dim = env.action_space.n

    # Hyperparameters
    num_rollouts = 8
    episode_length = 5000
    gamma = 0.9
    value_epochs = 4
    policy_epochs = 10
    batch_size = 32
    policy_batch_size = 256
    epsilon = 0.2
 
    # generate rollouts
    print("Epoch: " + str(e+1))
    rollouts = []
    steps = 0
    state = env.reset()
    print("Rollouts")
    for r in range(num_rollouts):
        print("Rollout:",r)
        cur_rollout = []
        state = env.reset()

        done = False
        while(not done):
            # rollout for a certain number of steps
            action = policy_net(format_state(state).cuda()).cpu().detach().numpy()
            action_id = np.argmax(np.random.multinomial(1, action.reshape((action_dim))))
            new_state, reward, done, _ = env.step(action_id)

            cur_rollout.append([resize(state), action.reshape(-1), action_id, reward])
            steps += 1
            state = new_state

        rollouts.append(cur_rollout)

    avg_steps = steps / num_rollouts
    print(avg_steps)
            
    # calculate the return for the rollouts
    calculate.returns(rollouts, gamma)

    # Approximate the value function
    value_dataset = AdvantageDataset(rollouts)
    value_loader = DataLoader(value_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    value_loader = value_loader
    value_losses = []
    print("Train Value")
    for _ in range(value_epochs):
        # train value network
        total_loss = 0
        for state, returns in value_loader:
            value_optim.zero_grad()
            returns = returns.unsqueeze(1).float()
            expected_returns = value_net(state.cuda())
            loss = value_loss(expected_returns, returns.cuda())
            loss.backward()
            total_loss += loss.cpu().item()
            value_optim.step()
        value_losses.append(total_loss)
        print("Value Loss", total_loss/value_epochs)
        
    # calculate the advantages for the rollouts 
    calculate.advantages(rollouts, value_net)
    
    del value_dataset

    # Learn a policy
    policy_dataset = PolicyDataset(rollouts)
    policy_loader = DataLoader(policy_dataset, batch_size=policy_batch_size, shuffle=True, pin_memory=True)
    policy_losses = []

    print("Train Policy")
    for _ in range(policy_epochs):
        # train policy network
        total_loss = 0
        for state, old_policy_dist, action_id, reward, return_reward, advantage in policy_loader:
            policy_optim.zero_grad()
            cur_batch_size = reward.size()[0]
            advantage = return_reward.float()

            new_policy_dist = policy_net(state.cuda()).cpu()

            ratio = new_policy_dist[range(cur_batch_size), action_id] / old_policy_dist[range(cur_batch_size), action_id] 

            left = ratio * advantage
            right = torch.clamp(ratio*advantage, 1 - epsilon, 1 + epsilon ) * advantage
            policy_loss = -torch.mean(torch.min(left, right))
            policy_loss.backward()
            total_loss += policy_loss.cpu().item()
            policy_optim.step()
            
            policy_losses.append(total_loss)
        print("Policy Loss", total_loss/policy_epochs)

    torch.save(policy_net.state_dict(), p_model_path)
    torch.save(value_net.state_dict(), v_model_path)

    return policy_net, policy_optim, policy_loss, value_net, value_optim, value_loss, avg_steps, np.mean(value_losses), np.mean(policy_losses)

In [0]:
def train_model(epochs = 50, test=False):

    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = gym.wrappers.Monitor(env, folder,force=True)

    action_dim = env.action_space.n

    state = env.reset()

    if os.path.exists(p_model_path):
        policy_net = Network(state_dim=img_size,action_dim=action_dim)
        policy_net.load_state_dict(torch.load(p_model_path))

        value_net = Network(state_dim=img_size)
        value_net.load_state_dict(torch.load(v_model_path))

        if test:
            return policy_net, env

    else:
        policy_net = Network(state_dim=img_size,action_dim=action_dim)
        value_net = Network(state_dim=img_size)

    policy_net = policy_net.cuda()
    value_net = value_net.cuda()
    policy_optim = optim.Adam(policy_net.parameters(), lr=1e-2, weight_decay=0.01)
    value_optim = optim.Adam(value_net.parameters(), lr=1e-3, weight_decay=1)
    value_loss = nn.MSELoss()
    policy_loss = 0

    epoch_losses = np.zeros((epochs,3))
    for e in range(epochs):
        policy_net, policy_optim, policy_loss, value_net, value_optim, value_loss, avg_steps, v_losses, p_losses  = train(e, env, policy_net, policy_optim, policy_loss, value_net, value_optim, value_loss)
        epoch_losses[e,:] = [avg_steps, v_losses, p_losses]

    plt.plot(epoch_losses[:,0])
    plt.title("Avg Steps")
    plt.savefig(step_graph)
    plt.show()

    plt.plot(epoch_losses[:,1])
    plt.title("Value Loss")
    plt.savefig(v_graph)
    plt.show()

    plt.plot(epoch_losses[:,2])
    plt.title("Policy Loss")
    plt.savefig(p_graph)
    plt.show()

    return policy_net, env



In [0]:
policy, env = train_model()

Epoch: 1
Rollouts
Rollout: 0


/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Rollout: 1
Rollout: 2
Rollout: 3
Rollout: 4
Rollout: 5
Rollout: 6


/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Rollout: 7
16589.875
Train Value
Value Loss 3030.884357985109
Value Loss 3005.964477499947
Value Loss 3018.252912676893
Value Loss 3005.504095816985
Train Policy
Policy Loss -27.6978567417711
Policy Loss -27.71456451378763
Policy Loss -27.717876727879048
Policy Loss -27.76019252538681
Policy Loss -27.74788079187274
Policy Loss -27.79940391276032
Policy Loss -27.752671939879654
Policy Loss -27.805617905035614
Policy Loss -27.756637449935077
Policy Loss -27.76128536127508
Epoch: 2
Rollouts
Rollout: 0
Rollout: 1
Rollout: 2
Rollout: 3
Rollout: 4
Rollout: 5
Rollout: 6
Rollout: 7
17830.375
Train Value
Value Loss 3026.913679065183
Value Loss 3028.1813949700445
Value Loss 2978.1921028494835
Value Loss 2958.345505528152
Train Policy
Policy Loss -24.16423335764557
Policy Loss -24.212078139185905
Policy Loss -24.161339030228554
Policy Loss -24.06666219374165
Policy Loss -24.12501909174025
Policy Loss -24.100251176580787
Policy Loss -24.13695824556053
Policy Loss -24.113887391611932
Policy Loss -2

...

18217.125
Train Value
Value Loss 2900.5571110527962
Value Loss 2852.449217265472
Value Loss 2824.7380643822253
Value Loss 2841.5671308264136
Train Policy
Policy Loss -22.52033277284354
Policy Loss -22.54813660569489
Policy Loss -22.55581676964648
Policy Loss -22.586310406401754
Policy Loss -22.60237948000431
Policy Loss -22.664696412719785
Policy Loss -22.566816040780395
Policy Loss -22.658725103363395
Policy Loss -22.58529151622206
Policy Loss -22.58885698989034
Epoch: 4
Rollouts
Rollout: 0
Rollout: 1
Rollout: 2
Rollout: 3
Rollout: 4
Rollout: 5
Rollout: 6
Rollout: 7
19780.0
Train Value
Value Loss 3053.58941411227
Value Loss 3027.1869280096143
Value Loss 3010.524067124352
Value Loss 2917.9898154158145
Train Policy
Policy Loss -19.42287497073412
Policy Loss -19.51370411850512
Policy Loss -19.469300531595945
Policy Loss -19.5134471103549
Policy Loss -19.67928682602942
Policy Loss -19.49982970878482
Policy Loss -19.695788002572954
Policy Loss -19.519042919017373
Policy Loss -19.5870470738

/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Rollout: 1
Rollout: 2
Rollout: 3
Rollout: 4
Rollout: 5
Rollout: 6
Rollout: 7
19430.25
Train Value
Value Loss 3379.0826885662973
Value Loss 3223.28149336949
Value Loss 3242.3252900559455
Value Loss 3259.255781468004
Train Policy
Policy Loss -23.0988364668563
Policy Loss -23.25601793024689
Policy Loss -23.102006452530624
Policy Loss -23.295649197883904
Policy Loss -23.182516769692302
Policy Loss -23.270656463690102
Policy Loss -23.095035139657558
Policy Loss -23.26186973005533
Policy Loss -23.201495306938888
Policy Loss -23.242001285776496
Epoch: 9
Rollouts
Rollout: 0
Rollout: 1
Rollout: 2
Rollout: 3
Rollout: 4
Rollout: 5
Rollout: 6
Rollout: 7
16073.375
Train Value
Value Loss 2814.4238365516067
Value Loss 2697.74330647476
Value Loss 2713.1218728534877
Value Loss 2726.6585514247417
Train Policy
Policy Loss -22.784914322011172
Policy Loss -22.941901949420572
Policy Loss -22.870045178383588
Policy Loss -22.86992472074926
Policy Loss -22.84001420568675
Policy Loss -22.91745574977249
Policy L

/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Rollout: 7
17052.625
Train Value
Value Loss 3041.5721716433764
Value Loss 3024.775761567056
Value Loss 3039.58279942628
Value Loss 3012.1719038318843
Train Policy
Policy Loss -27.9773456659168
Policy Loss -28.136329458095133
Policy Loss -28.09306268133223
Policy Loss -28.104107136279346
Policy Loss -28.108485999330878
Policy Loss -28.07700750492513
Policy Loss -28.0726795155555
Policy Loss -28.088499069213867
Policy Loss -28.046789860352874
Policy Loss -28.12728202007711
Epoch: 15
Rollouts
Rollout: 0
Rollout: 1
Rollout: 2
Rollout: 3
Rollout: 4
Rollout: 5
Rollout: 6
Rollout: 7
12171.375
Train Value
Value Loss 2979.0395720228553
Value Loss 2889.9591424055398
Value Loss 2871.2472942788154
Value Loss 2835.1297132540494
Train Policy
Policy Loss -34.349830695986746
Policy Loss -34.408799819648266
Policy Loss -34.392801500484346
Policy Loss -34.42260563969612
Policy Loss -34.43206027522683
Policy Loss -34.44048668462783
Policy Loss -34.38338605314493
Policy Loss -34.469825581088664
Policy Los

In [0]:
def run_model():
    policy_net, env = train_model(test=True)
    policy_net.eval()
    done = False
    state = env.reset()
    action_dim = env.action_space.n

    while(not done):
        # rollout for a certain number of steps
        action = policy_net(format_state(state)).detach().numpy()
        action_id = np.argmax(np.random.multinomial(1, action.reshape((action_dim))))
        state, reward, done, _ = env.step(action_id)


In [15]:
run_model()

/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
